In [1]:
# 사용자 입력 테스팅

age = int(input('나이?: '))
sex = input('성별? (M/F): ')
sex = sex.lower()
titles_to_recommend = []
while True:
  title = input('좋아하는 컨텐츠 (공백 입력 시 종료): ')
  if(not title): break
  titles_to_recommend.append(title)

나이?:  20
성별? (M/F):  M
좋아하는 컨텐츠 (공백 입력 시 종료):  언니네 산지직송2
좋아하는 컨텐츠 (공백 입력 시 종료):  


In [3]:
import pandas as pd

import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

## 1. 1차 후보군 생성: 좋아하는 콘텐츠 기반 (장르 기반 추천)

In [5]:
contents = pd.read_csv('./data/fixed_contents.csv')
contents = contents.fillna('')

In [6]:
# 다중값 컬럼을 리스트로 변환
preprocessing_contents = contents.copy()
multi_cols = ['genre_detail', 'director', 'platform', 'production', 'cast', 'country']
for col in multi_cols:
    preprocessing_contents[col] = contents[col].apply(
        lambda x: sorted(x.split(', '))
    )

preprocessing_contents.head()

,title,year,genre,genre_detail,director,runtime,platform,production,rating,broadcast_period,episodes,cast,country,language
0,당신의 맛,2025,드라마,"[로맨틱 코미디, 성장, 요리]",[박단희],,"[Genie TV, Netflix]",[쇼트케이크],,2025년 5월 12일 ~ 2025년 6월 10일,10부작,"[강하늘, 고민시, 김신록, 유수빈]",[대한민국],한국어
1,언젠가는 슬기로울 전공의생활,2025,드라마,"[로맨스, 성장, 의학, 일상, 청춘, 코미디, 휴먼]",[이민수],,"[Netflix, TVING]",[에그이즈커밍],,2025년 4월 12일 ~ 2025년 5월 18일,12부작,"[강유석, 고윤정, 신시아, 정준원, 한예지]",[대한민국],한국어
2,천국보다 아름다운,2025,드라마,"[가족, 로맨스, 블랙코미디, 추리, 판타지, 휴먼]",[김석윤],,[Netflix],"[SLL, 스튜디오 피닉스]",,2025년 4월 19일 ~ 2025년 5월 25일,12부작,"[김혜자, 류덕환, 손석구, 이정은, 천호진, 한지민]",[대한민국],한국어
3,귀궁,2025,드라마,"[가상역사극, 로맨틱 코미디, 액션, 퇴마, 판타지, 호러]","[김지연, 윤성식]",,"[Netflix, Wavve]","[스튜디오S, 아이윌미디어]",,2025년 4월 18일 ~ 2025년 6월 7일,16부작,"[김지연, 김지훈, 육성재]",[대한민국],한국어
4,약한영웅 Class 1,2022,드라마,"[느와르, 드라마, 범죄, 복수, 사회고발, 성장, 스릴러, 하이틴, 학원액션]",[유수민],,"[Netflix, Wavve]","[쇼트케이크, 플레이리스트]",,2023년 2월 17일 ~ 2023년 3월 10일,8부작,"[박지훈, 신승호, 이연, 최현욱, 홍경]",[대한민국],한국어


In [7]:
# 모든 텍스트 데이터를 하나의 필드로 결합
def create_soup(row):
  soup = ' '.join(row['genre_detail'])
  return soup

preprocessing_contents['soup'] = preprocessing_contents.apply(create_soup, axis=1)

In [8]:
# 모델과 토크나이저 불러오기
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 임베딩 벡터 계산 함수
def get_embeddings(texts):
    # 입력 텍스트를 토큰화
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        # 모델 출력 얻기
        outputs = model(**inputs)
        # 토큰 임베딩과 어텐션 마스크
        token_embeddings = outputs.last_hidden_state
        attention_mask = inputs["attention_mask"]

        # Mean Pooling 계산
        mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
        sum_embeddings = torch.sum(token_embeddings * mask_expanded, dim=1)
        sum_mask = mask_expanded.sum(dim=1)
        embeddings = sum_embeddings / sum_mask

    return embeddings

# 장르 텍스트를 임베딩 벡터로 변환
texts = preprocessing_contents['soup'].tolist()
embeddings = get_embeddings(texts)

# 코사인 유사도 계산
cosine_sim = F.cosine_similarity(embeddings.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)

print(cosine_sim)

tensor([[1.0000, 0.6996, 0.3675,  ..., 0.7075, 0.2772, 0.3729],
        [0.6996, 1.0000, 0.4508,  ..., 0.7415, 0.4118, 0.4300],
        [0.3675, 0.4508, 1.0000,  ..., 0.4290, 0.9273, 0.6289],
        ...,
        [0.7075, 0.7415, 0.4290,  ..., 1.0000, 0.3846, 0.5998],
        [0.2772, 0.4118, 0.9273,  ..., 0.3846, 1.0000, 0.6952],
        [0.3729, 0.4300, 0.6289,  ..., 0.5998, 0.6952, 1.0000]])


In [9]:
# 인덱스를 title로 설정
temp = contents.reset_index()
title_index = pd.Series(temp.index, index=temp['title']).drop_duplicates()

def get_recommendations(titles):
    # 유효한 제목들만 추출
    valid_indices = [title_index[title] for title in titles if title in title_index]

    if not valid_indices:
        return "입력된 제목 중 데이터셋에 존재하는 제목이 없습니다."

    # 선택한 제목들의 임베딩 벡터 추출
    selected_embeddings = embeddings[valid_indices]

    # 입력 제목들의 평균 벡터 계산
    mean_embedding = selected_embeddings.mean(dim=0)

    # 모든 콘텐츠와의 유사도 계산
    sim_scores = F.cosine_similarity(mean_embedding.unsqueeze(0), embeddings, dim=1)

    # 유사도 점수와 인덱스를 튜플로 묶고, 내림차순 정렬
    sim_scores = list(enumerate(sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 입력 제목 자체를 제외하고 상위 5개 추출
    top_scores = [score for score in sim_scores if score[0] not in valid_indices][:5]

    # 추천 영화의 인덱스와 유사도 점수 추출
    movie_indices = [i[0] for i in top_scores]
    similarity_scores = [i[1].item() for i in top_scores]

    # 추천 영화 제목과 유사도 점수 반환
    recommendations = contents['title'].iloc[movie_indices]

    # 결과를 데이터프레임으로 묶어서 반환
    result = pd.DataFrame({
        'title': recommendations,
        'similarity score': similarity_scores
    })

    return result

# 여러 개의 입력을 합쳐서 유사한 콘텐츠 5개 추천
contents_based_recommendations = get_recommendations(titles_to_recommend)
contents_based_recommendations['weight'] = range(5,0,-1)
contents_based_recommendations

,title,similarity score,weight
8,태어난 김에 세계일주4,0.872084,5
22,태어난 김에 세계일주 3,0.872084,4
29,지지고 볶는 여행,0.872084,3
52,태어난 김에 세계일주,0.872084,2
63,나 혼자 산다,0.671312,1


## 2. 2차 후보군 생성: 사용자 통계 기반 (사용자의 연령/성별 기반 추천)

In [11]:
user_data = None

# 성별에 따른 데이터 로드
if(sex=='m'):
  user_data = pd.read_csv('./data/daily_MALE_250514.csv')
else:
  user_data = pd.read_csv('./data/daily_FEMALE_250514.csv')
user_data = user_data.fillna('')

# 연령에 따른 데이터 필터링
if(age<20):
  user_data = user_data[user_data['age_group']=='10대'][['rank','title']]
elif(age<30):
  user_data = user_data[user_data['age_group']=='20대'][['rank','title']]
elif(age<40):
  user_data = user_data[user_data['age_group']=='30대'][['rank','title']]
elif(age<50):
  user_data = user_data[user_data['age_group']=='40대'][['rank','title']]
else:
  user_data = user_data[user_data['age_group']=='50대'][['rank','title']]

# 유저에 따른 콘텐츠 5개 추천
user_based_recommendations = user_data[:5]['title'].reset_index(drop=True).to_frame()
user_based_recommendations['weight'] = range(5,0,-1)
user_based_recommendations

,title,weight
0,당신의 맛,5
1,언젠가는 슬기로울 전공의생활,4
2,천국보다 아름다운,3
3,귀궁,2
4,약한영웅 Class 1,1


## 3. 후보군 통합

In [13]:
# 후보군을 위아래로 concat
recommendations = pd.concat([user_based_recommendations, contents_based_recommendations], ignore_index=True, sort=False)

# title이 겹치면 weight가 큰 컨텐츠를 남김
idx = recommendations.groupby('title')['weight'].idxmax()
recommendations = recommendations.loc[idx].reset_index(drop=True)

# title, weight, platform 정보를 남김
recommendations = recommendations[['title', 'weight']]
recommendations = recommendations.merge(preprocessing_contents[["title", "platform"]], on="title", how="left")
recommendations

,title,weight,platform
0,귀궁,2,"[Netflix, Wavve]"
1,나 혼자 산다,1,"[WATCHA, Wavve, coupang play]"
2,당신의 맛,5,"[Genie TV, Netflix]"
3,약한영웅 Class 1,1,"[Netflix, Wavve]"
4,언젠가는 슬기로울 전공의생활,4,"[Netflix, TVING]"
5,지지고 볶는 여행,3,[TVING]
6,천국보다 아름다운,3,[Netflix]
7,태어난 김에 세계일주,2,"[Wavve, coupang play]"
8,태어난 김에 세계일주 3,4,"[Netflix, Wavve]"
9,태어난 김에 세계일주4,5,"[Netflix, Wavve]"


## 4. 사용자 정보에 따라 ott 별로 가중치 부여

In [15]:
# 필요한 정보만 필터링
intentions = pd.read_csv('./data/OTT_유료서비스_계속_이용_의향__서비스별_20250413203427.csv', encoding='euc-kr')
intentions.columns = intentions.iloc[0]
intentions=intentions.loc[19:]

# 컬럼 이름 통일
intentions = intentions.rename(columns={"U+모바일 TV (%)": "U+모바일TV (%)"})

# 50대 이상은 하나로 분류
numeric_columns = intentions.columns[3:]
intentions[numeric_columns] = intentions[numeric_columns].astype(float)
sum_row = intentions.iloc[6:9, 3:].sum()
intentions.loc[intentions['구분별(2)'] == '50대', intentions.columns[3:]] = sum_row.values
intentions = intentions.iloc[:7]

intentions

,구분별(1),구분별(2),사례수 (가구원),웨이브 (%),티빙 (%),U+모바일TV (%),왓챠 (%),카카오TV (%),유튜브 (%),넷플릭스 (%),아프리카 TV (%),디즈니플러스 (%),쿠팡플레이 (%),애플TV+ (%),기타 (%),이용의향 모두 없음 (%),유료서비스 비이용 (%)
19,성별,남자,14554,2.0,2.6,0.1,0.9,0.1,8.0,18.7,0.2,2.1,1.7,0.1,0.0,2.0,73.5
20,성별,여자,14668,2.2,2.7,0.2,0.8,0.1,5.8,16.9,0.0,2.0,2.2,0.2,0.0,1.6,76.5
21,연령별,13~19세,1980,2.3,2.7,0.0,0.3,0.1,5.2,13.4,0.0,1.1,1.3,0.2,0.1,2.0,79.5
22,연령별,20대,4048,4.6,6.2,0.5,2.0,0.1,15.4,33.9,0.6,4.5,3.8,0.6,0.1,2.1,52.9
23,연령별,30대,4254,4.6,6.0,0.3,1.7,0.1,13.0,34.9,0.1,5.4,3.8,0.3,0.0,2.6,53.5
24,연령별,40대,4985,2.5,3.5,0.4,1.0,0.1,7.4,24.3,0.1,2.6,3.0,0.1,0.0,2.5,66.5
25,연령별,50대,5361,1.1,0.9,0.1,0.8,0.0,6.9,16.2,0.0,0.7,1.4,0.0,0.1,3.4,274.3


In [16]:
# 필요한 정보만 필터링
experiences = pd.read_csv('./data/OTT_이용_경험_여부_서비스별_20250413203230.csv', encoding='euc-kr')
experiences.columns = experiences.iloc[0]
experiences=experiences.loc[19:]

# 50대 이상은 하나로 분류
numeric_columns = experiences.columns[3:]
experiences[numeric_columns] = experiences[numeric_columns].astype(float)
sum_row = experiences.iloc[6:9, 3:].sum()
experiences.loc[experiences['구분별(2)'] == '50대', experiences.columns[3:]] = sum_row.values
experiences = experiences.iloc[:7]

experiences

,구분별(1),구분별(2),사례수 (가구원),유튜브 (%),넷플릭스 (%),웨이브 (%),티빙 (%),왓챠 (%),NOW (%),U+모바일TV (%),카카오TV (%),아프리카TV (%),디즈니플러스 (%),쿠팡플레이 (%),애플TV+ (%),기타 (%),OTT비이용 (%)
19,성별,남자,14554,64.0,32.4,4.8,6.0,2.4,1.0,0.8,1.1,1.9,4.0,3.8,0.4,0.0,31.8
20,성별,여자,14668,60.4,30.9,4.7,5.9,1.9,0.6,0.7,1.0,0.7,4.4,4.5,0.5,0.1,34.6
21,연령별,13~19세,1980,74.3,36.1,8.0,9.0,2.8,2.4,0.9,3.2,3.6,5.7,3.5,0.7,0.1,21.3
22,연령별,20대,4048,80.8,58.9,10.3,13.1,4.8,1.9,1.5,1.9,3.6,8.7,8.3,1.4,0.1,12.7
23,연령별,30대,4254,73.8,54.4,10.1,12.1,3.8,0.9,1.2,1.0,2.0,9.6,8.1,0.7,0.0,18.5
24,연령별,40대,4985,69.9,40.1,5.1,7.1,2.1,0.4,1.1,1.3,0.8,5.1,5.9,0.4,0.0,22.7
25,연령별,50대,5361,133.7,34.7,2.3,3.1,2.2,0.8,0.8,1.2,0.6,2.2,3.3,0.2,0.1,160.6


In [17]:
# 사용자 데이터에 맞는 row 추출
intentions_age_row = None
intentions_gender_row = None
experiences_age_row = None
experiences_gender_row = None

# 성별에 따른 데이터 필터링
if(sex=='m'):
  intentions_gender_row = intentions[intentions["구분별(2)"] == "남자"]
  experiences_gender_row = experiences[intentions["구분별(2)"] == "남자"]
else:
  intentions_gender_row = intentions[intentions["구분별(2)"] == "여자"]
  experiences_gender_row = experiences[intentions["구분별(2)"] == "여자"]

# 연령에 따른 데이터 필터링
if(age<20):
  intentions_age_row = intentions[intentions["구분별(2)"] == "13~19세"]
  experiences_age_row = experiences[intentions["구분별(2)"] == "13~19세"]
elif(age<30):
  intentions_age_row = intentions[intentions["구분별(2)"] == "20대"]
  experiences_age_row = experiences[intentions["구분별(2)"] == "20대"]
elif(age<40):
  intentions_age_row = intentions[intentions["구분별(2)"] == "30대"]
  experiences_age_row = experiences[intentions["구분별(2)"] == "30대"]
elif(age<50):
  intentions_age_row = intentions[intentions["구분별(2)"] == "40대"]
  experiences_age_row = experiences[intentions["구분별(2)"] == "40대"]
else:
  intentions_age_row = intentions[intentions["구분별(2)"] == "50대"]
  experiences_age_row = experiences[intentions["구분별(2)"] == "50대"]

In [18]:
# 점수를 저장할 딕셔너리
score_dict = {}

# OTT 서비스 리스트
ott_services = ["넷플릭스", "웨이브", "티빙", "왓챠", "U+모바일TV", "디즈니플러스", "쿠팡플레이", "애플TV+"]

# 가중치 설정
weight_age = 0.5
weight_gender = 0.5
weight_experience = 0.6
weight_intension = 0.4

# scaling을 위한 변수
max_score, min_score = 0, 1e9

# 종합 점수 계산
for ott in ott_services:
    # 의향 및 경험 데이터 추출
    intention_age = float(intentions_age_row[ott + " (%)"].values[0])
    experience_age = float(experiences_age_row[ott + " (%)"].values[0])
    intention_gender = float(intentions_gender_row[ott + " (%)"].values[0])
    experience_gender = float(experiences_gender_row[ott + " (%)"].values[0])

    # 각각의 종합 점수 계산
    score_age = weight_experience * experience_age + weight_intension * intention_age
    score_gender = weight_experience * experience_gender + weight_intension * intention_gender

    # 최종 종합 점수 계산
    final_score = (weight_age * score_age) + (weight_gender * score_gender)

    if(ott=='넷플릭스'):
      score_dict['Netflix']=final_score
    elif(ott=='웨이브'):
      score_dict['Wavve']=final_score
    elif(ott=='티빙'):
      score_dict['TVING']=final_score
    elif(ott=='왓챠'):
      score_dict['WATCHA']=final_score
    elif(ott=='U+모바일TV'):
      score_dict['U+모바일tv']=final_score
    elif(ott=='디즈니플러스'):
      score_dict['Disney+']=final_score
    elif(ott=='쿠팡플레이'):
      score_dict['coupang play']=final_score
    elif(ott=='애플TV+'):
      score_dict['Apple TV+']=final_score

    max_score = max(max_score, final_score)
    min_score = min(min_score, final_score)

# scaling (score가 1보다 작으면 weight에 곱해졌을 때 값이 작아지므로 최소값 1을 더함)
for k in score_dict:
  score_dict[k] = round((score_dict[k] - min_score) / (max_score - min_score) + 1, 2)

score_dict_adjusted = score_dict.copy()
penalty_factor = 0.6   # 원래 점수의 60%만 사용
score_dict_adjusted['Netflix'] = score_dict['Netflix'] * penalty_factor

score_dict_adjusted

{'Netflix': 1.2,
 'Wavve': 1.14,
 'TVING': 1.18,
 'WATCHA': 1.06,
 'U+모바일tv': 1.0,
 'Disney+': 1.12,
 'coupang play': 1.11,
 'Apple TV+': 1.0}

## 5. 추천된 컨텐츠를 기반한 최종적인 OTT 점수 계산

In [20]:
def get_ott_recommendation_ranking_normalized(recommendations, score_dict):
    # 플랫폼별 합계 점수와 개수를 구할 dict 초기화
    sum_score = {k:0 for k in score_dict}
    count = {k:0 for k in score_dict}

    # 합산
    for _, row in recommendations.iterrows():
        title, weight, plats = row['title'], row['weight'], row['platform']
        for ott in plats:
            if(ott=='Genie TV'): continue
            sum_score[ott] += score_dict[ott] * float(weight)
            count[ott] += 1

    # 평균화: sum / count
    avg_score = {}
    for ott in score_dict:
        if count[ott] > 0:
            avg_score[ott] = round(sum_score[ott] / count[ott], 2)
        else:
            avg_score[ott] = 0.0

    # DataFrame 생성 및 정렬
    df = pd.DataFrame({'OTT': list(avg_score.keys()),
                       'avg_score': list(avg_score.values())})
    return df.sort_values(by='avg_score', ascending=False).reset_index(drop=True)

get_ott_recommendation_ranking_normalized(recommendations, score_dict_adjusted)

,OTT,avg_score
0,TVING,4.13
1,Netflix,4.11
2,Wavve,2.85
3,coupang play,1.67
4,WATCHA,1.06
5,U+모바일tv,0.00
6,Disney+,0.00
7,Apple TV+,0.00
